# 使用Gluon接口来搭建线性回归模型

In [1]:
import mxnet as mx
import random
from mxnet import nd
import matplotlib as mpl
import matplotlib.pyplot as plt


C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 首先生成样本数据

In [6]:
#真实的模型
true_w = [2,-3.4]
true_b = 4.2
num_examples = 1000
num_inputs = 2
GPU = mx.gpu()

features = nd.random.normal(scale=1,shape=(num_examples,num_inputs),ctx=GPU)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += nd.random.normal(scale=0.01,shape=labels.shape,ctx=GPU)
    

## 之后从Gluon的data模块读取数据

In [7]:
from mxnet.gluon import data as gdata

In [8]:
batch_size = 10
dataset =gdata.ArrayDataset(features,labels)

In [11]:
for X,y in gdata.DataLoader(dataset,batch_size,shuffle=True):
       print(X)


[[ 0.23671432  0.16489032]
 [ 0.15244123 -0.25667492]
 [ 1.6910808  -1.4996583 ]
 [-1.010049    2.0362725 ]
 [ 2.2698884   0.4631495 ]
 [ 0.03778572  0.12671055]
 [-0.489199   -1.1253268 ]
 [ 2.8787382   1.5116961 ]
 [ 1.2737068   2.410616  ]
 [ 0.20657837  0.27739853]]
<NDArray 10x2 @gpu(0)>

[[-0.10694992  0.7149273 ]
 [ 1.852479    0.26228362]
 [ 0.24125496  0.880858  ]
 [ 0.16494763  0.8631748 ]
 [ 0.26472673 -1.4650618 ]
 [-0.06148157  0.6392977 ]
 [ 1.2559456  -1.2203082 ]
 [-0.7044006   0.8099297 ]
 [-0.49413183 -0.35872874]
 [-0.04811316  0.67053163]]
<NDArray 10x2 @gpu(0)>

[[ 0.5367312  -0.05952233]
 [-0.6994019   0.62754166]
 [ 0.00896326  0.4142605 ]
 [ 0.29261738  0.93103373]
 [ 0.22391176  0.04485791]
 [-0.23461635 -0.51844543]
 [-0.5502647  -0.22647358]
 [-0.84787047  0.0305592 ]
 [ 1.8006042   0.37353155]
 [ 2.8130758   0.5833393 ]]
<NDArray 10x2 @gpu(0)>

[[-0.8053443   0.9713582 ]
 [-0.10247385 -1.4361005 ]
 [-0.04118     0.30602005]
 [ 0.85211277  0.2609837 ]
 [-1.1

## 接下来利用gluon接口建立模型
```gluon.nn```表示定义了大量的神经网络层
+ 首先定义一个`nn.sequential`实例，在gluon中该实例可以看做是串联各个层的容器。在构造模型时，我们依次向该层中添加东西。给定输入数据时，该容器会自动将上一层的输出作为下一层的输入

In [12]:
from mxnet.gluon import nn
    net =nn.Sequential()

## 下面往构造好的容器中添加一个全连接层，其中输出维度为1,并且初始化模型参数

In [13]:
net.add(nn.Dense(1))

In [17]:
from mxnet import init
with GPU:
    net.initialize(init.Normal(sigma=0.01))

## 下面定义损失函数,平方损失函数其实就是L2Loss函数

In [15]:
import mxnet.gluon.loss as gloss
loss=gloss.L2Loss()

## 下面定义训练过程中的优化器
+ 使用`net.collect_param()`来收集训练过程中需要优化的参数

In [22]:
from mxnet import gluon
from mxnet import autograd
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

## 下面进行模型的训练

In [24]:
lr = 0.03
epochs = 3
with GPU:
    for epoch in range(1,epochs+1):
        #提取出训练的小批量数据
        for X,y in gdata.DataLoader(dataset,batch_size,shuffle=True):
            #前向传递并计算损失
            with autograd.record():
                l = loss(net(X),y)
            l.backward()              #求梯度
            trainer.step(batch_size)  #使用sgd进行优化
        l = loss(net(features),labels)
        print("epoch %d,loss %f"%(epoch,l.mean().asnumpy()))


epoch 1,loss 0.036441
epoch 2,loss 0.000135
epoch 3,loss 0.000050


## 查看训练完毕的模型

In [25]:
dense =net[0]
true_w,dense.weight.data()

([2, -3.4], 
 [[ 1.9996054 -3.399641 ]]
 <NDArray 1x2 @gpu(0)>)

In [26]:
true_b ,dense.bias.data()

(4.2, 
 [4.199242]
 <NDArray 1 @gpu(0)>)